# Fully Funtioning Backend Demo

This notebook is designed to show how the DSEM pipeline works. It's fully funtioning, using SEM and RL NAS. Please fasten your seat belt and enjoy the journey.

In [4]:
import numpy as np
import pandas as pd
import autogluon as ag
import rpy2.robjects as ro

from SEM import SemModel

## Data

In this demo, we use dataset _PoliticalDemocracy_ from Bollen's book on structural equation modeling published in 1989. We load this dataset in R and convert it to Python Dataframe for further usage.

In [2]:
# Load this dataset in R.
ro.packages.importr('lavaan')
rData = ro.r('PoliticalDemocracy')

# Convert it to Python Dataframe.
with ro.conversion.localconverter(ro.default_converter + ro.pandas2ri.converter):
    data = ro.conversion.rpy2py(rData)
data.describe()

,y1,y2,y3,y4,y5,y6,y7,y8,x1,x2,x3
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,5.464667,4.256443,6.563110,4.452533,5.136252,2.978074,6.196264,4.043390,5.054384,4.792195,3.557690
std,2.622702,3.947128,3.280891,3.349467,2.612602,3.372733,3.286240,3.245593,0.732904,1.510664,1.405711
min,1.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.784190,1.386294,1.001674
25%,2.900000,0.000000,3.766667,1.581500,3.691701,0.000000,3.477661,1.300916,4.477337,3.663233,2.300181
50%,5.400000,3.333333,6.666666,3.333333,5.000000,2.233333,6.666666,3.333333,5.075174,4.962845,3.568079
75%,7.500000,8.283332,9.999998,6.666666,7.500000,4.206853,9.999998,6.666666,5.515424,5.830362,4.522988
max,10.000000,9.999998,10.000000,10.000000,10.000000,9.999998,9.999998,10.000000,6.736967,7.872074,6.424591


## Conventional SEM

After loading this dataset, let's play with the conventional SEM. It is a truth universally acknowledged, that SEM is rather confirmative than explorative. So the very only way to play with the conventional SEM is to run SEM on a manually proposed model.

Therefore, we use the model from lavaan (a package in R) official tutorial as an example.

In [3]:
# Describe the model
model = '''
  # measurement model
    ind60 =~ x1 + x2 + x3
    dem60 =~ y1 + y2 + y3 + y4
    dem65 =~ y5 + y6 + y7 + y8
  # regressions
    dem60 ~ ind60
    dem65 ~ ind60 + dem60
  # residual correlations
    y1 ~~ y5
    y2 ~~ y4 + y6
    y3 ~~ y7
    y4 ~~ y8
    y6 ~~ y8
'''

# Create the model and fit the data.
sem = SemModel()
buildRes = sem.build_sem_model(model)
assert buildRes
fitRes = sem.fit_sem_model(data)
assert fitRes['is_fitted']

# Evaluate the model fitted.
measureRes = sem.evaluate_sem_model()
print(measureRes)

{'is_evaluated': True, 'agfi': 0.8541795802067569, 'rmsea': 0.034504482679424055}
